# ENSEMBLE MODEL TABLE OF CONTENT
- **Uploader**
- **Data Prep**
- **Categorical Data Prep and Modeling**
  - ADHD Model - Undersampled GBDT
  - Sex_F Model - Oversampled GBDT
- **Quantitative Data Prep and Modeling**
- **FCM Data Prep and Modeling**
- **Emsemble Modeling - Stacking Method**
- **Kaggle Submission Maker**

*Maya: I am up to getting best models for Cat rn, if anyone wants to find and input the best models along with their dataprep (dataset modifications) that would be great!*

*Kayla: I updated the categorical data prep and modeling bc we had an updated file with the fixed code after Preston pointed out our sampling mix up. I have also added the best models for quantitative data (even though I get different results than your separate colab for optuna ?)*

In [ ]:
#@title uploader
file_id = "1rVYGtWzw9s9Bw80-XaIN8gYBsBG79K8B" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

newwidsdatathon2025.zip
Archive:  newwidsdatathon2025.zip
replace Data Dictionary.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Data Prep

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install scikit-learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
#all datasets
train_quantitative = pd.read_excel('TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx')
train_answers = pd.read_excel('TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')
train_categorical = pd.read_excel("TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_solutions = pd.read_excel('TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')
fcm = pd.read_csv("TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
fcm_solutions = pd.read_excel("TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")

In [ ]:
#loading in datasets (I have it stored a different way)
train_categorical = pd.read_excel("/content/TRAIN_CATEGORICAL_METADATA.xlsx")
train_solutions = pd.read_excel("/content/TRAINING_SOLUTIONS.xlsx")
train_quantitative = pd.read_excel("/content/TRAIN_QUANTITATIVE_METADATA.xlsx")

## Categorical Data Prep + Modeling
### ADHD - Oversampling gradient boosted trees (gave worse accuracy?)


In [ ]:
#drop id
for feature in train_categorical.drop(columns = ['participant_id']).columns:
  print(train_categorical[feature].value_counts())
  print("\n")

Basic_Demos_Enroll_Year
2018    415
2019    312
2017    254
2016    191
2015     36
2020      5
Name: count, dtype: int64


Basic_Demos_Study_Site
1    652
3    430
4    120
2     11
Name: count, dtype: int64


PreInt_Demos_Fam_Child_Ethnicity
0.0    777
1.0    296
2.0     77
3.0     20
Name: count, dtype: int64


PreInt_Demos_Fam_Child_Race
0.0     573
8.0     195
1.0     181
2.0     128
3.0      30
9.0      23
10.0     11
4.0      10
11.0      6
7.0       2
Name: count, dtype: int64


MRI_Track_Scan_Location
2.0    532
3.0    463
1.0    179
4.0     36
Name: count, dtype: int64


Barratt_Barratt_P1_Edu
21.0    470
18.0    421
15.0    162
12.0     97
9.0      28
6.0      15
3.0       5
Name: count, dtype: int64


Barratt_Barratt_P1_Occ
0.0     286
35.0    219
45.0    187
30.0    154
40.0    154
25.0     79
15.0     35
5.0      31
20.0     31
10.0      6
Name: count, dtype: int64


Barratt_Barratt_P2_Edu
21.0    323
18.0    301
15.0    166
12.0    162
9.0      44
6.0      14
3.0       5

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')

participant_ids = df['participant_id']

In [ ]:
df.isna().sum()

,0
participant_id,0
Basic_Demos_Enroll_Year,0
Basic_Demos_Study_Site,0
PreInt_Demos_Fam_Child_Ethnicity,0
PreInt_Demos_Fam_Child_Race,0
MRI_Track_Scan_Location,0
Barratt_Barratt_P1_Edu,0
Barratt_Barratt_P1_Occ,0
Barratt_Barratt_P2_Edu,0
Barratt_Barratt_P2_Occ,0


In [ ]:
#fill missing NA values
df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)

In [ ]:
df.fillna({'PreInt_Demos_Fam_Child_Race':df['PreInt_Demos_Fam_Child_Race'].mean()}, inplace = True)

In [ ]:
df.fillna({'MRI_Track_Scan_Location':df['MRI_Track_Scan_Location'].mean()}, inplace = True)

In [ ]:
df.fillna({'Barratt_Barratt_P1_Edu':df['Barratt_Barratt_P1_Edu'].mean()}, inplace = True)

In [ ]:
df.fillna({'Barratt_Barratt_P1_Occ':df['Barratt_Barratt_P1_Occ'].mean()}, inplace = True)

In [ ]:
df.fillna({'Barratt_Barratt_P2_Edu':df['Barratt_Barratt_P2_Edu'].mean()}, inplace = True)

In [ ]:
df.fillna({'Barratt_Barratt_P2_Occ':df['Barratt_Barratt_P2_Occ'].mean()}, inplace = True)

In [ ]:
X = df.drop(columns=['Sex_F', 'participant_id'])
y = df['Sex_F']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rus = RandomUnderSampler(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

Running grid search got:

{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200} Validation accuracy: 0.7366255144032922

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

X_train_ADHD = X_train_resampled.drop(columns=['ADHD_Outcome'])
y_train_ADHD = X_train_resampled['ADHD_Outcome']
x_test_ADHD = X_test.drop(columns=['ADHD_Outcome'])
y_test_ADHD = X_test['ADHD_Outcome']

# Initialize GBDT model with best parameters
gbdt_cat_ADHD = GradientBoostingClassifier(
    learning_rate=0.01,
    max_depth=3,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=200,
    random_state=42
)

# Train the model
gbdt_cat_ADHD.fit(X_train_ADHD, y_train_ADHD)

# Evaluate on validation set
y_pred = gbdt_cat_ADHD.predict(x_test_ADHD)

# Accuracy
accuracy = accuracy_score(y_test_ADHD, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Weighted F1 Score
f1 = f1_score(y_test_ADHD, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

KeyError: "['ADHD_Outcome'] not found in axis"

### Sex_f - hybrid gradient boosting trees

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')

participant_ids = df['participant_id']

In [ ]:
#fill missing NA values
df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)

In [ ]:
X.isna().sum()

,0
Basic_Demos_Enroll_Year,0
Basic_Demos_Study_Site,0
PreInt_Demos_Fam_Child_Ethnicity,0
PreInt_Demos_Fam_Child_Race,54
MRI_Track_Scan_Location,3
Barratt_Barratt_P1_Edu,15
Barratt_Barratt_P1_Occ,31
Barratt_Barratt_P2_Edu,198
Barratt_Barratt_P2_Occ,222


In [ ]:
X = df.drop(columns=['Sex_F', 'ADHD_Outcome', 'participant_id']) #drop ADHD
y = df['Sex_F']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Best parameters: {'learning_rate': 0.05, 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500} Validation accuracy: 0.6213991769547325

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Initialize Gradient Boosting Classifier with best parameters
gbdt_cat_female = GradientBoostingClassifier(
    learning_rate=0.05,
    max_depth=7,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=500,
    random_state=42
)

# Train the model
gbdt_cat_female.fit(X_train_resampled, y_train_resampled)

# Predict on test set
y_pred = gbdt_cat_female.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Weighted F1 Score: {f1:.4f}")


Test Accuracy: 0.6337
Weighted F1 Score: 0.6435


##Quantitative Data Prep


### ADHD - Gradient Boosted DT

{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

In [ ]:
# Combine the quantitative data with the ADHD_Outcome labels
train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

# Separate features (X) and target variable (y)
X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['ADHD_Outcome']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the Gradient Boosting Classifier
gbdt_quant_ADHD = GradientBoostingClassifier(
    learning_rate=0.01,
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

gbdt_quant_ADHD.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gbdt_quant_ADHD.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Gradient Boosting Accuracy: {accuracy:.4f}')

f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

Gradient Boosting Accuracy: 0.7942
Weighted F1 Score: 0.7735


### Sex_f - Optuna Model

'n_estimators': 150, 'learning_rate': 0.08374413872051761, 'max_depth': 4


In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
# most code from geeks for geeks
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


# this code is taken from above ################
# Combine the quantitative data with the ADHD_Outcome labels
train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

# Separate features (X) and target variable (y)
X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['Sex_F']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler2 = StandardScaler()
X_train = scaler2.fit_transform(X_train)
X_test = scaler2.transform(X_test)
###################################################

def objective(trial):
    param_space = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
    }

    gb_model = GradientBoostingClassifier(**param_space, validation_fraction=0.1, n_iter_no_change=5, random_state=42)

    gb_model.fit(X_train, y_train)

    y_pred = gb_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return 1.0 - accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params_optuna = study.best_params
best_model_optuna = GradientBoostingClassifier(**best_params_optuna, validation_fraction=0.1, n_iter_no_change=5, random_state=42)
best_model_optuna.fit(X_train, y_train)

y_pred_best_optuna = best_model_optuna.predict(X_test)

accuracy_best_optuna = accuracy_score(y_test, y_pred_best_optuna)

print(f"Best Model Accuracy (Optuna): {accuracy_best_optuna}")


[I 2025-03-17 21:04:40,002] A new study created in memory with name: no-name-0c5f6b3a-1f6a-4560-883d-3185424c3316
[I 2025-03-17 21:04:40,091] Trial 0 finished with value: 0.2962962962962963 and parameters: {'n_estimators': 250, 'learning_rate': 0.13400074853499278, 'max_depth': 6}. Best is trial 0 with value: 0.2962962962962963.
[I 2025-03-17 21:04:40,203] Trial 1 finished with value: 0.28395061728395066 and parameters: {'n_estimators': 200, 'learning_rate': 0.057333729558506574, 'max_depth': 4}. Best is trial 1 with value: 0.28395061728395066.
[I 2025-03-17 21:04:40,251] Trial 2 finished with value: 0.3004115226337448 and parameters: {'n_estimators': 100, 'learning_rate': 0.1663484207562276, 'max_depth': 3}. Best is trial 1 with value: 0.28395061728395066.
[I 2025-03-17 21:04:40,301] Trial 3 finished with value: 0.29218106995884774 and parameters: {'n_estimators': 200, 'learning_rate': 0.16941702457235555, 'max_depth': 5}. Best is trial 1 with value: 0.28395061728395066.
[I 2025-03-17

Best Model Accuracy (Optuna): 0.7283950617283951


For some reason I am getting that the best parameters are {'n_estimators': 200, 'learning_rate': 0.11596113051499049, 'max_depth': 4} ?

In [ ]:
import optuna
# most code from geeks for geeks



# this code is taken from above ################
# Combine the quantitative data with the ADHD_Outcome labels
train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

# Separate features (X) and target variable (y)
X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['Sex_F']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler2 = StandardScaler()
X_train = scaler2.fit_transform(X_train)
X_test = scaler2.transform(X_test)
###################################################

gbdt_quant_female = GradientBoostingClassifier(
    learning_rate=0.11596113051499049,
    max_depth=4,
    n_estimators=200,
    validation_fraction=0.1,
    n_iter_no_change=5,
    random_state=42
)
gbdt_quant_female.fit(X_train, y_train)

y_pred_best_optuna = gbdt_quant_female.predict(X_test)

accuracy_best_optuna = accuracy_score(y_test, y_pred_best_optuna)

print(f"Best Model Accuracy (Optuna): {accuracy_best_optuna}")

f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")


Best Model Accuracy (Optuna): 0.7325102880658436
Weighted F1 Score: 0.2882


## FCM Data Prep

In [ ]:
!pip install torch-geometric

In [ ]:
#reset graph_fcm before running below cell
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np  # Ensure NumPy is imported

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming 'graph_fcm' DataFrame is already loaded and preprocessed
# Remove 'Sex_F' column
graph_fcm = graph_fcm.drop('Sex_F', axis=1)

# Prepare data for PyTorch Geometric
connectivity_columns = [col for col in graph_fcm.columns if col not in ['participant_id', 'ADHD_Outcome']]
features = graph_fcm[connectivity_columns].values
labels = graph_fcm['ADHD_Outcome'].values

# Create an adjacency matrix (replace with actual adjacency if available)
num_nodes = features.shape[0]  # Ensure correct shape (number of samples, not features)
adjacency_matrix = np.ones(((int)(num_nodes/2.5), (int)(num_nodes/2.5))) #LITERALLY JUST DIVIDED IN HALF SO ITS NOT FULLY CONNECTED ITS HALF CONNECTED?
np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

# Convert to PyTorch tensors and move to device
data = Data(
    x=torch.tensor(features, dtype=torch.float32).to(device),
    edge_index=torch.tensor(np.array(np.where(adjacency_matrix)), dtype=torch.long).to(device),
    y=torch.tensor(labels, dtype=torch.long).to(device)
)


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Hyperparameters
in_channels = data.num_features
hidden_channels = 64
out_channels = 2  # Two classes: ADHD (1) and no ADHD (0)
learning_rate = 0.01
epochs = 200

# Model, optimizer, and loss function
model_adhd = GCN(in_channels, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model_adhd.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
model_adhd.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model_adhd(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluation
model_adhd.eval()
_, pred = model_adhd(data.x, data.edge_index).max(dim=1)
correct = float(pred.eq(data.y).sum().item())
acc = correct / len(data.y)
print(f"Accuracy: {acc:.4f}")


In [ ]:
#reset graph_fcm before running below cell
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np  # Ensure NumPy is imported

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming 'graph_fcm' DataFrame is already loaded and preprocessed
# Remove 'Sex_F' column
graph_fcm = graph_fcm.drop('ADHD_Outcome', axis=1)

# Prepare data for PyTorch Geometric
connectivity_columns = [col for col in graph_fcm.columns if col not in ['participant_id', 'Sex_F']]
features = graph_fcm[connectivity_columns].values
labels = graph_fcm['Sex_F'].values

# Create an adjacency matrix (replace with actual adjacency if available)
num_nodes = features.shape[0]  # Ensure correct shape (number of samples, not features)
adjacency_matrix = np.ones(((int)(num_nodes/2.5), (int)(num_nodes/2.5)))
np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

# Convert to PyTorch tensors and move to device
data = Data(
    x=torch.tensor(features, dtype=torch.float32).to(device),
    edge_index=torch.tensor(np.array(np.where(adjacency_matrix)), dtype=torch.long).to(device),
    y=torch.tensor(labels, dtype=torch.long).to(device)
)


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Hyperparameters
in_channels = data.num_features
hidden_channels = 64
out_channels = 2  # Two classes: ADHD (1) and no ADHD (0)
learning_rate = 0.01
epochs = 200

# Model, optimizer, and loss function
model_f = GCN(in_channels, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model_f.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
model_f.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model_f(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluation
model_f.eval()
_, pred = model_f(data.x, data.edge_index).max(dim=1)
correct = float(pred.eq(data.y).sum().item())
acc = correct / len(data.y)
print(f"Accuracy: {acc:.4f}")


#Ensemble Modeling

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')
participant_ids = df['participant_id']

df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)

X = df.drop(columns=['Sex_F', 'participant_id'])
y = df['Sex_F']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rus = RandomUnderSampler(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

X_train_ADHD = X_train_resampled.drop(columns=['ADHD_Outcome'])
y_train_ADHD = X_train_resampled['ADHD_Outcome']
x_test_ADHD = X_test.drop(columns=['ADHD_Outcome'])
y_test_ADHD = X_test['ADHD_Outcome']

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')
participant_ids = df['participant_id']

df.fillna({'PreInt_Demos_Fam_Child_Ethnicity':df['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)

X = df.drop(columns=['Sex_F', 'ADHD_Outcome', 'participant_id']) #drop ADHD
y = df['Sex_F']

X_train, X_test_cat_female, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_resampled_female, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['Sex_F']

X_train_quant, X_test_quant, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Get base model predictions
pred_ADHD_1 = gbdt_cat_ADHD.predict_proba(X_train_ADHD)[:, 1]  # Probability of ADHD
pred_ADHD_2 = gbdt_quant_ADHD.predict_proba(X_train_quant)[:, 1]

pred_gender_1 = gbdt_cat_female.predict_proba(X_train_resampled_female)[:, 1]  # Probability of being male or female
pred_gender_2 = gbdt_quant_female.predict_proba(X_train_quant)[:, 1]

# Stack predictions as new features
meta_X_train_ADHD = np.column_stack((pred_ADHD_1, pred_ADHD_2[-688:]))
meta_X_train_female = np.column_stack((pred_gender_1[:970], pred_gender_2)) #I HAD TO CUT OFF THE FIRST MODEL BC THEY HAD DIFFERENT SIZES BUT THERE HAS TO BE A BETTER SOLUTION

# Define meta-targets
meta_y_train_adhd = train_solutions['ADHD_Outcome']  # True ADHD labels
meta_y_train_gender = train_solutions['Sex_F']  # True Gender labels


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.linear_model import LogisticRegression

# Meta-model for ADHD
meta_model_adhd = LogisticRegression()
meta_model_adhd.fit(meta_X_train_ADHD, meta_y_train_adhd[-688:])

# Meta-model for Gender
meta_model_gender = LogisticRegression()
meta_model_gender.fit(meta_X_train_female, meta_y_train_gender[:970])

LogisticRegression()

In [ ]:
# Get base model predictions on test data
pred_adhd_1_test = gbdt_cat_ADHD.predict_proba(x_test_ADHD)[:, 1]
pred_adhd_2_test = gbdt_quant_ADHD.predict_proba(X_test_quant)[:, 1]

pred_gender_1_test = gbdt_cat_female.predict_proba(X_test_cat_female)[:, 1]
pred_gender_2_test = gbdt_quant_female.predict_proba(X_test_quant)[:, 1]

# Stack predictions as new features
meta_X_test_ADHD = np.column_stack((pred_adhd_1_test, pred_adhd_2_test))
meta_X_test_gender = np.column_stack((pred_gender_1_test, pred_gender_2_test))


# Meta-model predictions
final_pred_adhd = meta_model_adhd.predict(meta_X_test_ADHD)
final_pred_gender = meta_model_gender.predict(meta_X_test_gender)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
# Accuracy for ADHD prediction
accuracy_adhd = accuracy_score(y_test_ADHD, final_pred_adhd)
print(f"Accuracy for ADHD prediction: {accuracy_adhd}")

# F1 Score for ADHD prediction
f1_adhd = f1_score(y_test_ADHD, final_pred_adhd, average='weighted')
print(f"F1 Score for ADHD prediction: {f1_adhd}")

# Accuracy for Gender prediction
accuracy_gender = accuracy_score(y_test, final_pred_gender)
print(f"Accuracy for Gender prediction: {accuracy_gender}")

# F1 Score for Gender prediction
f1_gender = f1_score(y_test, final_pred_gender, average='weighted')
print(f"F1 Score for Gender prediction: {f1_gender}")

Accuracy for ADHD prediction: 0.2674897119341564
F1 Score for ADHD prediction: 0.11290150179039067
Accuracy for Gender prediction: 0.7037037037037037
F1 Score for Gender prediction: 0.5813204508856683


In [ ]:
test_categorical = pd.read_excel("/content/TEST_CATEGORICAL.xlsx")
test_quantitative = pd.read_excel("/content/TEST_QUANTITATIVE_METADATA.xlsx")

In [ ]:
X_test_final_quant = test_quantitative.drop(['participant_id', 'MRI_Track_Age_at_Scan'], axis=1)
X_test_final_quant = X_test_final_quant.apply(lambda col: col.fillna(col.mean()), axis=0)

In [ ]:
X_test_final_cat = test_categorical.drop(['participant_id'], axis=1)
X_test_final_cat = X_test_final_cat.apply(lambda col: col.fillna(col.mode()[0]), axis=0)

In [ ]:
# Get base model predictions on test data
pred_adhd_1_test = gbdt_cat_ADHD.predict_proba(X_test_final_cat)[:, 1]
pred_adhd_2_test = gbdt_quant_ADHD.predict_proba(X_test_final_quant)[:, 1]

pred_gender_1_test = gbdt_cat_female.predict_proba(X_test_final_cat)[:, 1]
pred_gender_2_test = gbdt_quant_female.predict_proba(X_test_final_quant)[:, 1]

# Stack predictions as new features
meta_X_test_ADHD = np.column_stack((pred_adhd_1_test, pred_adhd_2_test))
meta_X_test_gender = np.column_stack((pred_gender_1_test, pred_gender_2_test))

# Meta-model predictions
final_pred_adhd = meta_model_adhd.predict(meta_X_test_ADHD)
final_pred_gender = meta_model_gender.predict(meta_X_test_gender)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
# Accuracy for ADHD prediction
accuracy_adhd = accuracy_score(y_test_ADHD, final_pred_adhd[:243])
print(f"Accuracy for ADHD prediction: {accuracy_adhd}")

# F1 Score for ADHD prediction
f1_adhd = f1_score(y_test_ADHD, final_pred_adhd[:243], average='weighted')
print(f"F1 Score for ADHD prediction: {f1_adhd}")

# Accuracy for Gender prediction
accuracy_gender = accuracy_score(y_test, final_pred_gender[:243])
print(f"Accuracy for Gender prediction: {accuracy_gender}")

# F1 Score for Gender prediction
f1_gender = f1_score(y_test, final_pred_gender[:243], average='weighted')
print(f"F1 Score for Gender prediction: {f1_gender}")

Accuracy for ADHD prediction: 0.2674897119341564
F1 Score for ADHD prediction: 0.11290150179039067
Accuracy for Gender prediction: 0.7037037037037037
F1 Score for Gender prediction: 0.5813204508856683


In [ ]:
participant_ids = test_quantitative['participant_id']

# Create a dictionary to hold the data
data = {
    'participant_id': participant_ids,
    'ADHD_Outcome': final_pred_adhd,
    'Sex_F': final_pred_gender
}

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('predictions.csv', index=False)

# Print the first few rows to verify (optional)
print(df.head(30))


   participant_id  ADHD_Outcome  Sex_F
0    Cfwaf5FX7jWK             0      0
1    vhGrzmvA3Hjq             0      0
2    ULliyEXjy4OV             0      0
3    LZfeAb1xMtql             0      0
4    EnFOUv0YK1RG             0      0
5    3VbkvJ22j9Fu             0      0
6    PRKZcnOgqcuk             0      0
7    DuVUuyMZi5qV             0      0
8    uM4etVLZrgMg             0      0
9    BpzyExrET5ta             0      0
10   sAqeb6F4lz97             0      0
11   u7XOOvHirIx7             0      0
12   aEPm4bEQvbYi             0      0
13   Fj9A5PWsIWKT             0      0
14   19mb5yGJigtw             0      0
15   v1nMpCoLGU0V             0      0
16   hRPuz4zpsEbw             0      0
17   mT8A6xa1O4Ro             0      0
18   4QBTjDoVpVt6             0      0
19   0X2H4LroxZcw             0      0
20   9CH7UxXuznUa             0      0
21   nU73zzjTnr4A             0      0
22   uEZHGukIUQ0k             0      0
23   jCzQwkpfgZyQ             0      0
24   Ljvrs76QJuI5        

Predicting everything is 0 ?

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Step 1: Load the test data
test_categorical = pd.read_excel('TEST/TEST_CATEGORICAL.xlsx')
test_quantitative = pd.read_excel('TEST/TEST_QUANTITATIVE_METADATA.xlsx')
test_fcm = pd.read_csv('TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')

# Step 2: Ensure test data has the same preprocessing as training data
# Get the same columns as used in training
cat_columns = [col for col in test_categorical.columns if col != 'participant_id']
quant_columns = [col for col in test_quantitative.columns if col != 'participant_id' and col != 'MRI_Track_Age_at_Scan']
fcm_columns = [col for col in test_fcm.columns if col != 'participant_id']

# Extract features from test data
test_cat_features = test_categorical[cat_columns]
test_quant_features = test_quantitative[quant_columns]
test_fcm_features = test_fcm[fcm_columns]

# Step 3: Apply imputation to handle missing values
# For categorical data
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_imputer.fit(test_cat_features)
test_cat_imputed = cat_imputer.transform(test_cat_features)

# For quantitative data
quant_imputer = SimpleImputer(strategy='mean')
quant_imputer.fit(test_quant_features)
test_quant_imputed = quant_imputer.transform(test_quant_features)

# Scale the quantitative data
scaler = StandardScaler()
scaler.fit(test_quant_imputed)
test_quant_scaled = scaler.transform(test_quant_imputed)

# For FCM data
fcm_imputer = SimpleImputer(strategy='mean')
fcm_imputer.fit(test_fcm_features)
test_fcm_imputed = fcm_imputer.transform(test_fcm_features)

# Step 4: Prepare FCM data for PyTorch Geometric
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_test_nodes = test_fcm_imputed.shape[0]
test_adjacency_matrix = np.ones(((int)(num_test_nodes/2.5), (int)(num_test_nodes/2.5)))
np.fill_diagonal(test_adjacency_matrix, 0)  # No self-loops

# Convert to PyTorch Geometric format
test_fcm_data = Data(
    x=torch.tensor(test_fcm_imputed, dtype=torch.float32).to(device),
    edge_index=torch.tensor(np.array(np.where(test_adjacency_matrix)), dtype=torch.long).to(device)
)

# Step 5: Make predictions using each model
# Categorical model predictions for ADHD
cat_adhd_pred = gbdt_cat_ADHD.predict(test_cat_imputed)

# Categorical model predictions for Sex_F
cat_sex_pred = gbdt_cat_female.predict(test_cat_imputed)

# Quantitative model predictions for ADHD
quant_adhd_pred = gbdt_quant_ADHD.predict(test_quant_scaled)

# Quantitative model predictions for Sex_F
quant_sex_pred = gbdt_quant_female.predict(test_quant_scaled)

# FCM model predictions for ADHD
model_adhd.eval()
with torch.no_grad():
    fcm_adhd_logits = model_adhd(test_fcm_data.x, test_fcm_data.edge_index)
    _, fcm_adhd_pred = fcm_adhd_logits.max(dim=1)
    fcm_adhd_pred = fcm_adhd_pred.cpu().numpy()

# FCM model predictions for Sex_F
model_f.eval()
with torch.no_grad():
    fcm_sex_logits = model_f(test_fcm_data.x, test_fcm_data.edge_index)
    _, fcm_sex_pred = fcm_sex_logits.max(dim=1)
    fcm_sex_pred = fcm_sex_pred.cpu().numpy()

# Step 6: Create ensemble predictions using majority voting
# For ADHD prediction
adhd_votes = np.vstack([cat_adhd_pred, quant_adhd_pred, fcm_adhd_pred])
adhd_majority = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=adhd_votes)

# For Sex_F prediction
sex_votes = np.vstack([cat_sex_pred, quant_sex_pred, fcm_sex_pred])
sex_majority = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=sex_votes)

# Step 7: Create final prediction dataframe
final_pred = pd.DataFrame({
    'participant_id': test_fcm['participant_id'].values,
    'ADHD_Outcome': adhd_majority,
    'Sex_F': sex_majority
})

# Print summary statistics
adhd_count = final_pred['ADHD_Outcome'].sum()
no_adhd_count = len(final_pred) - adhd_count
female_count = final_pred['Sex_F'].sum()
male_count = len(final_pred) - female_count

print(f"ADHD: {adhd_count}, No ADHD: {no_adhd_count}")
print(f"Female: {female_count}, Male: {male_count}")

# Save predictions to CSV
final_pred.to_csv('predictions.csv', index=False)
from google.colab import files
files.download('predictions.csv')

# Verify the format
print(final_pred.head())

ADHD: 304, No ADHD: 0
Female: 111, Male: 193


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  participant_id  ADHD_Outcome  Sex_F
0   Cfwaf5FX7jWK             1      0
1   vhGrzmvA3Hjq             1      1
2   ULliyEXjy4OV             1      0
3   LZfeAb1xMtql             1      1
4   EnFOUv0YK1RG             1      0


Predicting all ADHD for some reason

In [ ]:
# Get probabilities from quantitative model (since it showed some variation)
quant_adhd_proba = gbdt_quant_ADHD.predict_proba(test_quant_scaled)[:, 1]

# Instead of majority voting or all 1's, set a low threshold to get ~85-90% ADHD predictions
# Try a threshold of 0.5 or lower to get most but not all as ADHD
threshold = 0.4  # Adjust this value to control the percentage of ADHD predictions
adhd_pred = (quant_adhd_proba > threshold).astype(int)

# Calculate the percentage of ADHD predictions with this threshold
adhd_percent = np.mean(adhd_pred) * 100
print(f"Threshold {threshold}: ADHD = {adhd_percent:.1f}% of participants")

# Alternative approach: Take the bottom N% by probability and set them to 0
# This gives you direct control over the percentage
percentage_nonADHD = 15  # Set this to control what percentage are predicted as non-ADHD
cutoff_value = np.percentile(quant_adhd_proba, percentage_nonADHD)
adhd_pred_alt = (quant_adhd_proba > cutoff_value).astype(int)
adhd_percent_alt = np.mean(adhd_pred_alt) * 100
print(f"Bottom {percentage_nonADHD}%: ADHD = {adhd_percent_alt:.1f}% of participants")

# Use the alternative approach for final predictions
final_pred = pd.DataFrame({
    'participant_id': test_fcm['participant_id'].values,
    'ADHD_Outcome': adhd_pred_alt,  # This will predict ADHD for 85% of participants
    'Sex_F': sex_majority  # Keep the sex predictions as they were
})

# Print summary statistics
adhd_count = final_pred['ADHD_Outcome'].sum()
no_adhd_count = len(final_pred) - adhd_count
female_count = final_pred['Sex_F'].sum()
male_count = len(final_pred) - female_count

print(f"ADHD: {adhd_count}, No ADHD: {no_adhd_count}")
print(f"Female: {female_count}, Male: {male_count}")

# Save predictions to CSV
final_pred.to_csv('predictions.csv', index=False)
from google.colab import files
files.download('predictions.csv')

# Verify the format
print(final_pred.head())

got 0.66 from above